<a href="https://colab.research.google.com/github/luisadosch/Final-Project-snapAddy/blob/main/model6_Bag_of_Words_TF%E2%80%93IDF_%2B_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Github-Zugangsdaten

In [53]:
# GitHub-Zugangsdaten
import pandas as pd

GH_USER = "luisadosch"
GH_REPO = "Final-Project-snapAddy"
BRANCH = "main"

def get_github_url(relative_path):
    return f"https://raw.githubusercontent.com/{GH_USER}/{GH_REPO}/{BRANCH}/{relative_path}"


jobs_annotated_active_df = pd.read_csv(get_github_url("data/processed/jobs_annotated_active.csv"))

department_df = pd.read_csv(get_github_url("data/raw/department-v2.csv"))

seniority_df = pd.read_csv(get_github_url("data/raw/seniority-v2.csv"))

# 2. Modell Seniority

In [54]:
#Seniority Daten sortieren
sdf = seniority_df.copy()

sdf["text"] = sdf["text"].astype(str).str.lower()
sdf["label"] = sdf["label"].astype(str)

sdf = sdf.dropna(subset=["text", "label"])

Prepare the seniority dataset for modeling. Lowercasing ensures uniform text representation. Dropping missing values prevents errors in the model.

In [67]:
#Seniority Train/Test Split
from sklearn.model_selection import train_test_split

sx = sdf["text"]
sy = sdf["label"]

sx_train, sx_test, sy_train, sy_test = train_test_split(
    sx,
    sy,
    test_size=0.2,
    random_state=42,
    stratify=sy
)

# Print dataset sizes
print("Seniority dataset sizes:")
print("Total:", len(sx))
print("Train:", len(sx_train))
print("Test:", len(sx_test))

Seniority dataset sizes:
Total: 9428
Train: 7542
Test: 1886


Split into training and test sets. stratify ensures rare classes are represented proportionally.

In [56]:
#Seniority TF–IDF + Logistic Regression Pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

smodel = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),   # unigrams + bigrams
        min_df=3,
        max_df=0.9
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])

Pipeline converts job titles into TF–IDF features and applies logistic regression. class_weight="balanced" ensures rare seniority levels get enough importance.

In [57]:
#Seniority Modell trainieren
smodel.fit(sx_train, sy_train)

# Vorhersagen auf Testdaten
sy_pred = smodel.predict(sx_test)

# Accuracy ausgeben
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(sy_test, sy_pred))

Accuracy: 0.9703075291622482


Train the seniority classifier and generate predictions on the test set. The model achieves a high accuracy of 0.97 on the test set.

In [58]:
#Seniority Evaluation
from sklearn.metrics import accuracy_score, f1_score, classification_report

sy_pred = smodel.predict(sx_test)

saccuracy = accuracy_score(sy_test, sy_pred)
smacro_f1 = f1_score(sy_test, sy_pred, average="macro")

print("Accuracy:", round(saccuracy, 3))
print("Macro F1:", round(smacro_f1, 3))
print("\nClassification Report:\n")
print(classification_report(sy_test, sy_pred))

Accuracy: 0.97
Macro F1: 0.956

Classification Report:

              precision    recall  f1-score   support

    Director       0.99      0.98      0.98       197
      Junior       0.85      1.00      0.92        82
        Lead       0.97      0.98      0.98       709
  Management       0.92      0.93      0.92       151
      Senior       0.99      0.97      0.98       747

    accuracy                           0.97      1886
   macro avg       0.94      0.97      0.96      1886
weighted avg       0.97      0.97      0.97      1886



Evaluate using accuracy and macro F1, which accounts for class imbalance. Classification report shows precision, recall, and F1 per seniority level. The evaluation yields an accuracy of 0.97 and a macro F1 score of 0.956, reflecting strong performance across all seniority classes.

In [ ]:
#CV Daten evaluieren

In [59]:
#Seniority Top Features
import numpy as np

feature_names = smodel.named_steps["tfidf"].get_feature_names_out()
coefs = smodel.named_steps["clf"].coef_

for i, label in enumerate(smodel.named_steps["clf"].classes_):
    top = np.argsort(coefs[i])[-10:]
    print(f"\nTop words for {label}:")
    print(feature_names[top])


Top words for Director:
['marketing director' 'managing directors' 'managing' 'director marketing'
 'vertriebsdirektor' 'director sales' 'directors' 'sales director'
 'abteilungsdirektor' 'director']

Top words for Junior:
['marketing' 'assistent' 'associate' 'assistentin' 'mitarbeiter'
 'mitarbeiterin' 'referent' 'referentin' 'analyst' 'junior']

Top words for Lead:
['abteilungsleiter' 'projektleiter' 'geschäftsleitung' 'teamleiter'
 'leiterin' 'head of' 'head' 'vertriebsleiter' 'leitung' 'leiter']

Top words for Management:
['cio' 'vice president' 'vice' 'founder' 'chief' 'owner' 'ceo'
 'geschäftsführung' 'vp' 'geschäftsführer']

Top words for Senior:
['marketing manager' 'engineer' 'executive' 'assistant' 'managerin'
 'responsable' 'consultant' 'management' 'senior' 'manager']


Shows the most influential words for predicting each seniority class. Helps interpret the model’s decisions.

3. Modell Department

In [60]:
#Department Daten sortieren
ddf = department_df.copy()

ddf["text"] = ddf["text"].astype(str).str.lower()
ddf["label"] = ddf["label"].astype(str)

ddf = ddf.dropna(subset=["text", "label"])

In [61]:
# Department Train/Test Split
from sklearn.model_selection import train_test_split

dx = ddf["text"]
dy = ddf["label"]

dx_train, dx_test, dy_train, dy_test = train_test_split(
    dx,
    dy,
    test_size=0.2,
    random_state=42,
    stratify=dy
)

print(len(dx))
print(len(dy))
print(len(dx_train))
print(len(dy_train))
print(len(dx_test))
print(len(dy_test))

10145
10145
8116
8116
2029
2029


In [62]:
# Department TF–IDF + Logistic Regression Pipeline
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

dmodel = Pipeline([
    ("tfidf", TfidfVectorizer(
        ngram_range=(1, 2),   # unigrams + bigrams
        min_df=3,
        max_df=0.9
    )),
    ("clf", LogisticRegression(
        max_iter=1000,
        class_weight="balanced"
    ))
])

In [63]:
# Department Modell trainieren
dmodel.fit(dx_train, dy_train)

# Vorhersagen auf Testdaten
dy_pred = dmodel.predict(dx_test)

# Accuracy ausgeben
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(dy_test, dy_pred))

Accuracy: 0.9344504682109414


In [64]:
# Department Evaluation
from sklearn.metrics import accuracy_score, f1_score, classification_report

dy_pred = dmodel.predict(dx_test)

daccuracy = accuracy_score(dy_test, dy_pred)
dmacro_f1 = f1_score(dy_test, dy_pred, average="macro")

print("Accuracy:", round(daccuracy, 3))
print("Macro F1:", round(dmacro_f1, 3))
print("\nClassification Report:\n")
print(classification_report(dy_test, dy_pred))

Accuracy: 0.934
Macro F1: 0.86

Classification Report:

                        precision    recall  f1-score   support

        Administrative       0.62      0.94      0.74        17
  Business Development       0.83      0.99      0.90       124
            Consulting       0.82      0.97      0.89        33
      Customer Support       0.88      1.00      0.93         7
       Human Resources       0.75      1.00      0.86         6
Information Technology       0.92      0.95      0.94       261
             Marketing       0.99      0.92      0.96       859
                 Other       0.50      1.00      0.67         8
    Project Management       0.57      0.88      0.69        40
            Purchasing       0.89      1.00      0.94         8
                 Sales       0.96      0.93      0.94       666

              accuracy                           0.93      2029
             macro avg       0.79      0.96      0.86      2029
          weighted avg       0.95      0.93   

In [65]:
# Department Top Features
import numpy as np

feature_names = dmodel.named_steps["tfidf"].get_feature_names_out()
coefs = dmodel.named_steps["clf"].coef_

for i, label in enumerate(dmodel.named_steps["clf"].classes_):
    top = np.argsort(coefs[i])[-10:]
    print(f"\nTop words for {label}:")
    print(feature_names[top])


Top words for Administrative:
['assistentin des' 'geschäftsführung' 'gf' 'assistent der'
 'geschäftsleitung' 'der' 'sekretärin' 'assistent' 'assistenz'
 'assistentin']

Top words for Business Development:
['business intelligence' 'crm' 'digital business' 'of business'
 'business process' 'ebusiness' 'it business' 'development'
 'business development' 'business']

Top words for Consulting:
['senior berater' 'sap' 'coach' 'senior consultant' 'von' 'senior'
 'recruitment' 'beraterin' 'berater' 'consultant']

Top words for Customer Support:
['service and' 'it systems' 'customer' 'technical' 'it support' 'it'
 'customer support' 'technical support' 'supporter' 'support']

Top words for Human Resources:
['qualitätsmanagement' 'director digital' 'project director' 'gl'
 'manager hr' 'of human' 'resources' 'human resources' 'human' 'hr']

Top words for Information Technology:
['digitalization' 'administrator' 'entwickler' 'digitale' 'administration'
 'digitalisierung' 'sap' 'digital' 'it' 'cr

In [66]:
# Vergleiche Accuracy und Macro F1
comparison_metrics = pd.DataFrame({
    "Target": ["Seniority", "Department"],
    "Accuracy": [saccuracy, daccuracy],
    "Macro F1": [smacro_f1, dmacro_f1]
})

print("Modellvergleich:\n")
print(comparison_metrics)

# Optional: Top 5 Features pro Label für beide Modelle nebeneinander
def print_top_features(model, n=5):
    feature_names = model.named_steps["tfidf"].get_feature_names_out()
    coefs = model.named_steps["clf"].coef_
    for i, label in enumerate(model.named_steps["clf"].classes_):
        top = np.argsort(coefs[i])[-n:]
        print(f"\nTop {n} words for {label}:")
        print(feature_names[top])

print("\n--- Seniority Top Features ---")
print_top_features(smodel)

print("\n--- Department Top Features ---")
print_top_features(dmodel)

Modellvergleich:

       Target  Accuracy  Macro F1
0   Seniority  0.970308  0.956030
1  Department  0.934450  0.860444

--- Seniority Top Features ---

Top 5 words for Director:
['director sales' 'directors' 'sales director' 'abteilungsdirektor'
 'director']

Top 5 words for Junior:
['mitarbeiterin' 'referent' 'referentin' 'analyst' 'junior']

Top 5 words for Lead:
['head of' 'head' 'vertriebsleiter' 'leitung' 'leiter']

Top 5 words for Management:
['owner' 'ceo' 'geschäftsführung' 'vp' 'geschäftsführer']

Top 5 words for Senior:
['responsable' 'consultant' 'management' 'senior' 'manager']

--- Department Top Features ---

Top 5 words for Administrative:
['der' 'sekretärin' 'assistent' 'assistenz' 'assistentin']

Top 5 words for Business Development:
['ebusiness' 'it business' 'development' 'business development' 'business']

Top 5 words for Consulting:
['senior' 'recruitment' 'beraterin' 'berater' 'consultant']

Top 5 words for Customer Support:
['it' 'customer support' 'technical su